<a href="https://colab.research.google.com/github/svecx/Report-Script/blob/staging/Final_Branch_Master_SAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
  import pandas as pd
  from datetime import datetime

In [8]:
def load_data():
    df_master = pd.read_excel("190825_BRANCH_MASTER_SAP.XLSX", keep_default_na=False)
    df_lookup = pd.read_excel("DWH_BRANCH 20250731.xlsx", sheet_name="020825", keep_default_na=False)
    return df_master, df_lookup

def merge_data(df_master, df_lookup):
    df_result = pd.merge(
        df_master,
        df_lookup[['SUBBR', 'MBNAME', 'RGNAME']],
        left_on="Branch ID - Originating Branch of Accoun",
        right_on="SUBBR",
        how="left"
    )
    df_result[['MBNAME', 'RGNAME']] = df_result[['MBNAME', 'RGNAME']].fillna("#NA")
    df_result = df_result[
        ['Branch ID - Originating Branch of Accoun','Single-Character Flag','Location of the Branch',
         'Revenue Owner','Business Segment','Changed by','Changed On','Time of change','MBNAME','RGNAME']
    ]
    return df_result


In [9]:
def generate_summary(df_master, df_result):
    filter_1 = df_master.groupby(
        ['Single-Character Flag','Revenue Owner','Business Segment'], dropna=False
    ).size().reset_index(name='Jumlah')

    filter_2 = filter_1[
        (filter_1['Single-Character Flag'] == 'U') & (
            (filter_1['Revenue Owner'].isna() | (filter_1['Revenue Owner'] == '')) |
            (filter_1['Business Segment'].isna() | (filter_1['Business Segment'] == ''))
        )
    ].shape[0]

    T_Filter3 = df_result[df_result['MBNAME'] == '#NA'].shape[0]
    T_Filter4 = df_result[df_result['RGNAME'] == '#NA'].shape[0]

    summary = pd.DataFrame({
        'Nama': ['Filter 2', 'T_Filter3', 'T_Filter4'],
        'Jumlah': [filter_2, T_Filter3, T_Filter4]
    })

    return summary, (filter_2, T_Filter3, T_Filter4)


def validate(filters):
    filter_2, T_Filter3, T_Filter4 = filters
    validations = [
        ("Filter_2 harus < 1", filter_2 < 1),
        ("T_Filter3 harus = 12", T_Filter3 == 12),
        ("T_Filter4 harus = 12", T_Filter4 == 12),
    ]
    invalids = [f"❌ {msg}" for msg, condition in validations if not condition]

    if invalids:
        print("📌 Masalah ditemukan:\n" + "\n".join(invalids))
    else:
        print("✅ Semua kondisi terpenuhi, data aman.")


In [10]:
def save_result(df_result):
    today = datetime.today().strftime('%d-%m-%Y')
    filename = f"Merge Branch Master Sap {today}.xlsx"
    df_result.to_excel(filename, index=False)
    print(f"📂 File berhasil disimpan: {filename}")


In [11]:
df_master, df_lookup = load_data()
df_result = merge_data(df_master, df_lookup)
summary, filters = generate_summary(df_master, df_result)

print(summary)
validate(filters)
save_result(df_result)


        Nama  Jumlah
0   Filter 2       0
1  T_Filter3      12
2  T_Filter4      12
✅ Semua kondisi terpenuhi, data aman.
📂 File berhasil disimpan: Merge Branch Master Sap 19-08-2025.xlsx


In [14]:
df_result.head()

,Branch ID - Originating Branch of Accoun,Single-Character Flag,Location of the Branch,Revenue Owner,Business Segment,Changed by,Changed On,Time of change,MBNAME,RGNAME
0,1,B,LCTN10,,,BTCADM,2025-08-19,07:00:24,00001 -- KC Ambon (Konsolidasi-MB) ...,P -- Makassar ...
1,10,B,LCTN17,,,BTCADM,2025-08-19,07:00:24,00010 -- KC Blora (Konsolidasi-MB) ...,G -- Semarang ...
2,100,B,LCTN1,,,BTCADM,2025-08-19,07:00:24,00100 -- KC Tasikmalaya (Konsolidasi-MB) ...,F -- Bandung ...
3,1001,S,LCTN15,,,BTCADM,2025-08-19,07:00:24,00059 -- KC Palembang A. Rivai (Konsolidasi-MB...,D -- Palembang ...
4,1006,S,LCTN17,,,BTCADM,2025-08-19,07:00:24,00083 -- KC Semarang Patimura (Konsolidasi-MB)...,G -- Semarang ...


# Pengecekan Error

---



In [12]:
filter_1 = df.groupby(['Single-Character Flag','Revenue Owner','Business Segment'], dropna=False).size().reset_index(name='Jumlah')
filter_1
filter_2 = filter_1[(filter_1['Single-Character Flag'] == 'U')]
# filter_2 = filter_1[
#     (filter_1['Single-Character Flag'] == 'U') &
#     ((filter_1['Revenue Owner'] == 140.0) & (filter_1['Business Segment'] == 'M'))
# ]
filter_2

,Single-Character Flag,Revenue Owner,Business Segment,Jumlah
8,U,140,M,5387
